## LDA Preprocessing 1
# Artifact Removal and Stopword Selection
This notebook takes the clean JSON files for each article and does some preprocessing to obtain a text that we can analyze using LDA.

Specifically, we do:
* Artifact removal
* Stopword selection

In the next notebook we will do:
* Punctuation removal
* Lemmatization

In [18]:
import re

We will also use a utility `Corpus` class defined in the `utils/` folder.

The `Corpus` class includes methods to get the documents, which we get as `Article` objects.

In [62]:
from utils.corpus import Corpus

## Loading the corpus

In [64]:
corpus = Corpus(registry_path='utils/article_registry.json')

utils/article_registry.json
9


In [21]:
corpus.documents[0].type

'PRESENTACIÓN'

We will only work with documents in Spanish, with text, and that are indeed articles (and not reviews, for example). The `Corpus` class included a method with these criteria by default.

In [22]:
corpusList = corpus.get_documents_list()

This leaves is with approximately 700 articles.

In [23]:
len(corpusList) 

8

Let's check for duplicates in case we have some. This is important in case we downloaded an article twice, once in PDF and once in HTML.

In [24]:
len(set([doc.id for doc in corpusList]))

8

## Artifact removal

There are some artifacts included in the text that are produced by HTML and PDF processing.

Let's start by removing numbers and some special characters such as newline characters (`\n`). We will keep normal punctuation for now as that might help SpaCy when we do lemmatization.

In [25]:
for doc in corpusList:
    doc.clean_text = re.sub('\d|\n',' ', doc.text)

We can detect some of these artifacts by looking for non-alphanumeric characters between alphanumeric characters (e.g. `"ar-gument"`, `"ar\xadgument"`).

In [26]:
artifacts = re.compile('\w+[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\d\s:]\w+')

In [27]:
[re.findall(artifacts, doc.text) for doc in corpusList][0]

['nd_naturallaw_forum',
 'hf_l',
 'xiii_enc_15051891_rerum',
 'hist_councils',
 'ii_vatican_council',
 'ii_decl_19651207_dignitatis',
 'humanae_en',
 'hf_p',
 'xi_enc_19310515_quadragesimo']

One common artifact is the hex `\xad` for the soft hyphen which is used to break lines. We can remove it easily.

In [28]:
for doc in corpusList:
    doc.clean_text = re.sub('\\xad','', doc.clean_text)
    doc.clean_text = doc.clean_text.replace(u"\xa0", "")

We can save the corpus for now.

In [29]:
corpus.save_documents()

## Stopword Selection
Stopword removal is perhaps the most difficult part of preprocessing. There are two challenges to meet:
* Some stopword lists such as the one included in NLTK for Spanish are too weak and do not filter many stopwords.
* Other stopword lists are too inclusive and can eliminate words that are meaningful in philosophy (e.g. 'verdadero', true). 
It is important to note that stopwords are very context-sensitive. A word in one context may provide little meaning (hence counting as a stopword) while in other contexts it may provide lots of information.

To tackle these challenges, we will first to an initial filtering with NTLK's list. This will leave many stopwords in the text, but will reduce the size of each text considerably. Then we will compare the text with a stronger list of stopwords (source). We will see which words are both in the text and the stronger stopwords list. We will inspect these lists manually and extract a list of protected words. We will iterate over this process a number of times. Once we have a robust list of protected words, we will concatenate NLTK's stopwords list with the stronger one and eliminate the protected words from it. This will provide a final (hopefully middle ground) stopword list with which to continue.

In [30]:
from nltk.corpus import stopwords as nltk_stopwords

stopwords_weak = nltk_stopwords.words('spanish')

In [31]:
import requests

r = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-es/master/stopwords-es.txt')
stopwords_strong = r.text.split('\n')

In [32]:
document_words = []
for doc in corpusList:
    document_words += [word for word in re.findall('\w+', doc.clean_text) if word not in stopwords_weak]
document_words[:10]

In [34]:
from collections import Counter

docwords_and_stopwords = Counter([word for word in document_words if word in stopwords_strong])

In [35]:
docwords_and_stopwords.most_common(10)

[('x', 1120),
 ('ser', 135),
 ('puede', 130),
 ('dos', 107),
 ('parte', 105),
 ('si', 102),
 ('solo', 99),
 ('hace', 91),
 ('manera', 91),
 ('sino', 90)]

Already in the first 10 most common words in both the documents and the strong list of stopwords we find words that in philosophy are quite meaningful:
* 'ser': being
* 'bien': good
* 'modo': mode

We will start saving those words and eliminating them from the stronger list of stopwords. Then we will repeat the process of selecting the words that are in both lists and see which words are common. By iterating over this process a couple of times, we will get a list of protected words.

In [52]:
from IPython.display import clear_output

protected_words = ['bien']
new_protected_words = ['ser']
checked_words = []

while new_protected_words != "None":
    docwords_and_stopwords = Counter([word for word in document_words if word in stopwords_strong and word not in checked_words])
    checked_words += [word for word, count in docwords_and_stopwords.most_common(20)]
    print(docwords_and_stopwords.most_common(20))
    new_protected_words = input("New protected words (comma separated) [end with 'None']: ")
    protected_words += new_protected_words.split(', ')
    clear_output()

Additionally, we have added words that we observed were incorrectly lemmatized. We will pass the list of protected words to the lemmatizer later on and we will skip these protected words.

In [22]:
protected_words = [word for word in protected_words if word and word != 'None']
protected_words = list(set(protected_words))

### Removing stopwords in other languages

Given that most of the articles have abstracts in English, some of the usual stopwords in English are appearing frequently in our documents. Thus, we will append the NLTK-generated list of English stopwords. We will also use one for Portuguese, which we sometimes get as well.

In [59]:
english_stopwords = nltk_stopwords.words("english")
portuguese_stopwords = nltk_stopwords.words("portuguese")

TODO: should we be filtering these just like we filter the Spanish ones?

### Other stopwords custom to our corpus

There are some other stopwords that we would like to include, but that have not been taken into account in the previous processes. These are found in `wordlists/custom_stopwords.txt`.

Once we have a robust set of words we can save both the final stopword list and the protected words list.

In [60]:
with open('wordlists/custom_stopwords.txt') as file:
    custom_stopwords = file.read().split()

stopwords_final = list(set(
    stopwords_weak + stopwords_strong + english_stopwords + portuguese_stopwords +  custom_stopwords
))
with open('wordlists/stopwords.txt', 'w') as fp:
    fp.write('\n'.join(stopwords_final))

with open('wordlists/protectedWords.txt', 'w') as fp:
    fp.write('\n'.join(protected_words))

In [30]:
corpus.save_documents()

# Final replacements and edits
PDF correction is not perfect and we observe some artifacts left in the LDA. A hotfix is to do those replacements manually for now and check whether we can improve on this process in the future.

Note: I saved these and removed the cells which contained this dictionary. We can find it in `wordlists/old_manual_replacements.json`.